# Applying data minimization to a trained ML model

In this tutorial we will show how to perform data minimization for ML models using the minimization module. 

This will be demonstarted using the Adult dataset (original dataset can be found here: https://archive.ics.uci.edu/ml/datasets/adult). 

We use only the numerical features in the dataset because this is what is currently supported by the module.

## Load data

In [1]:
import numpy as np

# Use only numeric features (age, education-num, capital-gain, capital-loss, hours-per-week)
x_train = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
                        usecols=(0, 4, 10, 11, 12), delimiter=", ")

y_train = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
                        usecols=14, dtype=str, delimiter=", ")


x_test = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
                        usecols=(0, 4, 10, 11, 12), delimiter=", ", skiprows=1)

y_test = np.loadtxt("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
                        usecols=14, dtype=str, delimiter=", ", skiprows=1)

# Trim trailing period "." from label
y_test = np.array([a[:-1] for a in y_test])

y_train[y_train == '<=50K'] = 0
y_train[y_train == '>50K'] = 1
y_train = y_train.astype(np.int)

y_test[y_test == '<=50K'] = 0
y_test[y_test == '>50K'] = 1
y_test = y_test.astype(np.int)

print(x_train)

[[3.9000e+01 1.3000e+01 2.1740e+03 0.0000e+00 4.0000e+01]
 [5.0000e+01 1.3000e+01 0.0000e+00 0.0000e+00 1.3000e+01]
 [3.8000e+01 9.0000e+00 0.0000e+00 0.0000e+00 4.0000e+01]
 ...
 [5.8000e+01 9.0000e+00 0.0000e+00 0.0000e+00 4.0000e+01]
 [2.2000e+01 9.0000e+00 0.0000e+00 0.0000e+00 2.0000e+01]
 [5.2000e+01 9.0000e+00 1.5024e+04 0.0000e+00 4.0000e+01]]


## Train decision tree model

In [2]:
import os
import sys
sys.path.insert(0, os.path.abspath('..'))

from apt.utils.datasets import ArrayDataset
from apt.utils.models import SklearnClassifier, ModelOutputType
from sklearn.tree import DecisionTreeClassifier

base_est = DecisionTreeClassifier()
model = SklearnClassifier(base_est, ModelOutputType.CLASSIFIER_PROBABILITIES)
model.fit(ArrayDataset(x_train, y_train))

print('Base model accuracy: ', model.score(ArrayDataset(x_test, y_test)))

Base model accuracy:  0.8190528837295007


## Run minimization
We will try to run minimization with different possible values of target accuracy (how close to the original model's accuracy we want to get, 1 being same accuracy as for original data).

In [4]:
from apt.minimization import GeneralizeToRepresentative
from sklearn.model_selection import train_test_split

# default target_accuracy is 0.998
minimizer = GeneralizeToRepresentative(model)

# Fitting the minimizar can be done either on training or test data. Doing it with test data is better as the 
# resulting accuracy on test data will be closer to the desired target accuracy (when working with training 
# data it could result in a larger gap)
# Don't forget to leave a hold-out set for final validation!
X_generalizer_train, x_test, y_generalizer_train, y_test = train_test_split(x_test, y_test, stratify=y_test,
                                                                test_size = 0.4, random_state = 38)
x_train_predictions = model.predict(ArrayDataset(X_generalizer_train))
if x_train_predictions.shape[1] > 1:
    x_train_predictions = np.argmax(x_train_predictions, axis=1)
minimizer.fit(dataset=ArrayDataset(X_generalizer_train, x_train_predictions))
transformed = minimizer.transform(dataset=ArrayDataset(x_test))

print('Accuracy on minimized data: ', model.score(ArrayDataset(transformed, y_test)))

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): 0.920665
Improving accuracy
feature to remove: 1
Removed feature: 1, new relative accuracy: 0.920026
feature to remove: 0
Removed feature: 0, new relative accuracy: 0.938580
feature to remove: 4
Removed feature: 4, new relative accuracy: 0.987204
feature to remove: 2
Removed feature: 2, new relative accuracy: 0.992962
feature to remove: 3
Removed feature: 3, new relative accuracy: 1.000000
Accuracy on minimized data:  0.8165771297006907


#### Let's see what features were generalized

In [5]:
generalizations = minimizer.generalizations
print(generalizations)

{'ranges': {}, 'categories': {}, 'untouched': ['2', '4', '3', '1', '0']}


We can see that for the default target accuracy of 0.998 of the original accuracy, no generalizations are possible (all features are left untouched, i.e., not generalized).

Let's change to a slightly lower target accuracy.

In [6]:
# We allow a 1% deviation in accuracy from the original model accuracy
minimizer2 = GeneralizeToRepresentative(model, target_accuracy=0.99)

minimizer2.fit(dataset=ArrayDataset(X_generalizer_train, x_train_predictions))
transformed2 = minimizer2.transform(dataset=ArrayDataset(x_test))
print('Accuracy on minimized data: ', model.score(test_data=ArrayDataset(transformed2, y_test)))
generalizations2 = minimizer2.generalizations
print(generalizations2)

Initial accuracy of model on generalized data, relative to original model predictions (base generalization derived from tree, before improvements): 0.920665
Improving accuracy
feature to remove: 1
Removed feature: 1, new relative accuracy: 0.920026
feature to remove: 0
Removed feature: 0, new relative accuracy: 0.938580
feature to remove: 4
Removed feature: 4, new relative accuracy: 0.987204
feature to remove: 2
Removed feature: 2, new relative accuracy: 0.992962
Accuracy on minimized data:  0.8100537221795856
{'ranges': {'3': [704.0, 782.0, 870.0, 951.0, 1588.0, 1647.5, 1684.0, 1805.0, 1923.0, 2168.5]}, 'categories': {}, 'untouched': ['2', '4', '1', '0']}


This time we were able to generalize one feature, feature number 3 (capital-loss).